# Compare Brinkman and Thema thesaurus
In this document we try to find the amount of overlap between two thesauri: The Brinkman Catalog (used by the KB) and Thema (created by Centraal Bookhuis (CB)). This list will be used to choose our dataset for Annif.

In [1]:
# !pip3 install numpy
# !pip3 install pandas

import pandas as pd
import numpy as np

In [2]:
# Import TSV file for Thema and Brinkman as Series.
brinkman_data = pd.read_csv('data/brinkmanthesaurus_vocab.tsv', sep='\t', usecols=[1], header=None, squeeze=True)
thema_data = pd.read_csv('data/Thema_v1.3.2_nl.tsv', sep='\t', usecols=[1], header=None, squeeze=True)

In [12]:
# Sort data and print lenth.
brinkman_data.sort_values(ascending=True, inplace=True)
thema_data.sort_values(ascending=True, inplace=True)
print('Total Brinkman subjects:\t{}'.format(len(brinkman_data)))
print('Total Thema subjects:\t\t{}'.format(len(thema_data)))
# brinkman_data.head(10)
# thema_data.head(10)

Total Brinkman subjects:	14737
Total Thema subjects: 		7362


In [4]:
# Intersection

# Case and punctuation mark sensitive comparison (e.g. 's Gravenhage, s gravenhage).
intersect_direct = pd.Series(list(set(thema_data).intersection(set(brinkman_data))))
print('Total common subjects (exact matches):\t{}'.format(len(intersect_direct)))

# Case incesitive comparison (convert all entries to lowercase before comparison).
intersect_lower = pd.Series(list(set(thema_data.str.lower()).intersection(set(brinkman_data.str.lower()))))
print('\nTotal matches when converting to lowercase first:\t{}\n'.format(len(intersect_lower)))
print(intersect_lower.head(10))

# Is Capitalization important to these listed subjects? Can we use the lowercase version intersection from here on out?
# To determine if a mapping could be done 1-on-1 between Brinkman and Thema we want to see if capitalization is important.

# Matches after converted to lowercase.
intersect_lower_uniq = pd.Series(list((set(intersect_lower) - set(intersect_direct.str.lower()))))

print('\n{} subjects that are in both lists but have different capitalization:\n'.format(len(intersect_lower_uniq)))

print(intersect_lower_uniq.head(10))

Total common subjects (exact matches):	668

Total matches when converting to lowercase first:	980

0                 landbouweconomie
1                            blues
2                       kalimantan
3    internationaal belastingrecht
4                            dubai
5                           patmos
6                           bhutan
7      geestelijke gezondheidszorg
8                  elektrotechniek
9                             mons
dtype: object

312 subjects that are in both lists but have different capitalization:

0                 landbouweconomie
1                 kennismanagement
2                            blues
3                            barok
4                       ontbossing
5    internationaal belastingrecht
6                       sociologie
7                economische groei
8                   impressionisme
9                  elektrotechniek
dtype: object


In [5]:
# Unique subjects
# What makes these subjects unique?

brinkman_uniq = pd.Series(list(set(brinkman_data.str.lower()) - set(thema_data.str.lower())))
print('Total unique subjects in Brinkman:\t{}\n'.format(len(brinkman_uniq)))

print(brinkman_uniq.head(10))

Total unique subjects in Brinkman:	13729

0        kattenbroek (amersfoort)
1                     encyclieken
2                       wespelaar
3           nederlandse dialecten
4                memphis (egypte)
5                       remmerden
6             kronieken (i en ii)
7    rollecatekwartier (deventer)
8                          boksum
9                        de venen
dtype: object


In [6]:
thema_uniq = pd.Series(list(set(thema_data.str.lower()) - set(brinkman_data.str.lower())))
print('Total unique subjects in Thema:\t\t{}\n'.format(len(thema_uniq)))
print(thema_uniq.head(10))

Total unique subjects in Thema:		6355

0                                           shintoïsme
1                                   ideeëngeschiedenis
2     economie, financiën, bedrijfsleven en management
3    voor duitse beroepskwalificaties als specialis...
4                           loch lomond & de trossachs
5                                       psychosomatiek
6    children’s / teenage general interest: time tr...
7                                             la rioja
8                                     himachal pradesh
9                              mondelinge geschiedenis
dtype: object


In [7]:
# Search what 'autisme' (a subject only in Brinkman) is in Thema ('autisme' is a very common subject).
thema_filter = thema_uniq[thema_uniq.str.contains('autisme')]
thema_filter

2421    autisme en asperger syndroom
4165     omgaan met autisme/asperger
dtype: object

In [8]:
# Is the subject 'cleveland' in Thema equal as 'cleveland (gbr.)' in Brinkman?
brinkman_filter = brinkman_uniq[brinkman_uniq.str.contains('cleveland')]
brinkman_filter

9874        old cleveland
10985    cleveland (gbr.)
dtype: object

In [9]:
# Een mogelijke volgende stap zou kunnen zijn om:
#   - Keuze maken tussen aan subset aan onderwerpen om verder te gaan.
#   - Zelf onderwerpen gaan groeperen/mappen (bv. in synoniemen) (of bv. 'omgaan met autisme/asperger' en 'autisme en asperger syndroom' groeperen onder 'autisme'). om zo veel mogelijk onderwerpen te kunnen gebruiken om te trainen?


# Het lijkt erop dat je hoofletters wel weg wil halen,die 668 exacte matches zijn plaatsnamen ed.
# Brinkman gebruikt hoofdletters voor (plaats)namen, maar niet voor andere woorden.
# Thema doet dit ook, maar begint ook elk subject met een hoofdletter.
# print(intersect_direct)

# Het lijkt erop dat we misschien het beste de onderwerpen kunnen gebruiken die matchen nadat ze waren geconverteerd.
# intersect_lower

# - Hoeveel onderwerpen willen we gaan testen?
# - Focussen op alleen (plaats)namen die exact matchten (668 onderwerpen - intersect_direct),
#           of een mix van alle matchende woorden (zelf een handjevol uitkiezen),
#           de volle mep gebruiken (980 onderwerpen - intersect_lower),
#           of alleen onderwerpen die past matchen nadat ze geconverteerd waren naar lowercase (312 onderwerpen - intersect_lower_uniq)?


# Op basis hiervan kunnen bepalen welke onderwerpen we gaan gebruiken om - na het automatisch toewijzing van een onderwerp uit Thema - te controleren of de juiste onderwerpen zijn gekozen.
# Onze aankomende testset met ebooks filteren op alleen deze onderwerpen? (bias?)

In [10]:
# Export 980 overlapping words to text file
intersect_lower.to_csv(r'/Users/haighton_macbook/Desktop/brinkman_thema.txt', header=None, index=None, sep=' ', mode='a')

In [11]:
# We need the PPN (other column which was not imported)
df = pd.read_csv('data/brinkmanthesaurus_vocab.tsv', sep='\t', header=None, squeeze=True)

df_ppn = df[df.iloc[:, 1].str.lower().isin(intersect_lower)]
df_ppn = (df_ppn.iloc[:, 0])
df_ppn = df_ppn.str.split('/').str[-1].str.rsplit('>').str[0]
df_ppn.to_csv(r'/Users/haighton_macbook/Desktop/brinkman_thema_ppn.txt', header=None, index=None, sep=' ', mode='a')